In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import optuna
import matplotlib.pyplot as plt

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')
sub_df = pd.read_csv('/kaggle/input/playground-series-s5e8/sample_submission.csv')

# RAW DATA  With optuna XGBoost
 {'max_depth': 10, 'learning_rate': 0.088542375425159, 'n_estimators': 353, 'subsample': 0.7933680125492807, 'colsample_bytree': 0.665859932215261, 'gamma': 1.9591038948726576, 'reg_alpha': 4.135658777432998, 'reg_lambda': 3.0378978111339086}

# RAW DATA  With optuna XGBoost bez cechy previous 
 {'max_depth': 9, 'learning_rate': 0.11410300715659989, 'n_estimators': 368, 'subsample': 0.907977121199552, 'colsample_bytree': 0.6014268962223818, 'gamma': 0.7012459904827175, 'reg_alpha': 4.361523501536162, 'reg_lambda': 4.294305690785547}




In [ ]:
# ==== WPROWADŹ SWOJE KOLUMNY ====
categorical_cols = [ 'job', 'marital','education','default','housing','loan','contact','month','poutcome']  # <- wpisz swoje kolumny kategoryczne
numerical_cols = ['age','balance', 'day', 'duration', 'campaign','pdays']# , 'previous']    # <- wpisz swoje kolumny numeryczne USUŃ prevous dla 2 modelu
target_col = 'y'

In [ ]:
# ==== WCZYTAJ DANE ====
# train_df i test_df muszą być wcześniej wczytane

# RAW DATA  With optuna XGBoost without previous

best_params =   {'max_depth': 8, 'learning_rate': 0.07733762308533498, 'n_estimators': 469, 'subsample': 0.9190881144681935, 'colsample_bytree': 0.7461531023839627, 'gamma': 2.06998675412871, 'reg_alpha': 1.3641260353935682, 'reg_lambda': 4.343504135311625}

X = train_df[categorical_cols + numerical_cols].copy()
y = train_df[target_col].copy()
X_test =  test_df[categorical_cols + numerical_cols].copy()

# ==== ENKODOWANIE DANYCH KATEGORYCZNYCH ==== Dla CATBOOSTA MOŻNA POMINĄ TYLKO PODAĆ LISTĘ
# for col in categorical_cols:
#     le = LabelEncoder()
#     X[col] = le.fit_transform(X[col])
#     X_test[col] = le.transform(X_test[col])

for col in ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]:
    X[col] = X[col].astype("category")
    X_test[col] =  X_test[col].astype("category")

final_model = XGBClassifier(**best_params, enable_categorical=True) # wpisz jeszcze banance oraz auc!!!!!!!!!!!!!!!!!!!
final_model.fit(X, y)

# Predykcja na testowym zbiorze (np. do submission)
y_pred = final_model.predict_proba(X_test)[:, 1]



In [ ]:
submission = pd.DataFrame({
    "id": test_df["id"],        # lub np. range(len(y_pred_final))
    "Personality": y_pred
})

submission.to_csv("submission.csv", index=False)
print('Sukcesss!!!')
